In [1]:
import cv2
import os 
from cvzone.HandTrackingModule import HandDetector
import numpy as np

In [ ]:
width = 1280
height = 780

cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)


folderPath = r'C:\Users\HP\Desktop\New folder\Images'
pathImage = sorted(os.listdir(folderPath), key=len)
print(pathImage)


imgNumaber = 0
ws, hs = int(width*0.3), int(height*0.3)
gestureThreshold = int(width/2)
buttonPressed = False
buttonCounter = 0
buttonDelay = 15
annotations = [[]]
annotationNumber = 0
annotationStart = False

detector = HandDetector(detectionCon=0.8, maxHands=1)

while True:
    
    success, img = cap.read()
    img = cv2.flip(img, 1)
    pathFullImage = os.path.join((folderPath), pathImage[imgNumaber])
    imgCurrent = cv2.imread(pathFullImage)
    
    hands, img = detector.findHands(img)
    print(annotations)
    if hands and buttonPressed is False:
        hand = hands[0]
        fingers = detector.fingersUp(hand)
        cx, cy = hand['center']
        lmlist = hand['lmList']
        
        xVal = int(np.interp(lmlist[8][0], [ws, ws*2], [0, width*2]))
        yVal = int(np.interp(lmlist[8][1], [hs,hs*2], [0, height*2]))
        
        indexFinger = xVal, yVal
        
        if cy<= gestureThreshold:
            
            annotationStart == False
            
            # Gesture-1
            if fingers == [1, 0, 0, 0, 0]:
                annotationStart = False
                if imgNumaber > 0:
                    buttonPressed = True
                    annotations = [[]]
                    annotationNumber = 0
                    imgNumaber -= 1
                    
            # Gesture-2
            if fingers == [0, 0, 0, 0, 1]:
                annotationStart = False
                if imgNumaber < (len(pathImage)-1):
                    buttonPressed = True
                    annotations = [[]]
                    annotationNumber = 0
                    imgNumaber += 1
                    
        # Gesture-3
        if fingers == [0, 1, 0, 0, 0]:
            cv2.circle(imgCurrent, indexFinger, 17, (0,0,255), cv2.FILLED)
            annotationStart == False
                    
        # Gesture-4
        if fingers == [0, 1, 1, 0, 0]:
            if annotationStart is False:
                annotationStart == True
                annotationNumber += 1
                annotations.append([])
            cv2.circle(imgCurrent, indexFinger, 25, (0,0,255), cv2.FILLED)
            annotations[annotationNumber].append(indexFinger)
        
        else:
            annotationStart = False    
        
        # Gesture-5
        if fingers == [1, 1, 1, 1, 1]:
            if annotations:
                if annotationNumber >= 0:
                    annotations.pop(-1)
                    annotationNumber -= 1
                    buttonPressed = True
        
    else:
        annotationStart == False
    
    if buttonPressed:
        buttonCounter += 1
        if buttonCounter > buttonDelay:
            buttonCounter = 0
            buttonPressed = False
    
    for i in range(len(annotations)):
        for j in range(len(annotations[i])):
            if j !=0 :
                cv2.line(imgCurrent, annotations[i][j-1], annotations[i][j], (0,0,200), 15)
    
    imgSmall = cv2.resize(img, (ws, hs))
    imgCurrent = cv2.resize(imgCurrent, (width, height))
    h, w, _ = imgCurrent.shape
    
    imgCurrent[0 : hs, (w-ws) : w]  = imgSmall
    
    cv2.imshow('Slides', imgCurrent)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break    
    
    